In [1]:
import warnings
warnings.filterwarnings("ignore")

from data_loader import DataLoader
from models import MODELS
from misc.evaluation_metrics import *
from attacks import *

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

In [2]:
dl = DataLoader()
info = dl.get_info()

display(info)

,name,frequency,description,source,path
0,Electricity Transformer Data - 15 min,fifteen minutes,...,https://github.com/zhouhaoyi/ETDataset,data/electricity_transformer_15min.csv
1,Metro Interstate Human Traffic Volume,hour,Hourly Interstate 94 Westbound traffic volume ...,https://archive.ics.uci.edu/ml/datasets/Metro+...,data/Metro_Interstate_Traffic_Volume.csv
2,Beijing-Guanyuan Air-Quality,hour,This hourly data set considers 6 main air poll...,https://archive.ics.uci.edu/ml/datasets/Beijin...,data/beijing_air_quality_Data_Guanyuan_2013030...
3,Solar Generation - EnerjiSA,hour,Solar generation data of unlicenced solar cen...,https://www.kaggle.com/competitions/enerjisa-e...,data/enerjisa_solar_generation.csv


In [3]:
df = dl.load(3)
(X_train, y_train, X_test, y_test), scaler = dl.prepare_dataset(23*4,1)

print(dl.last_loaded_info["name"])
print(X_train.shape)
print(X_test.shape)
print()

# model = MODELS[-1]((X_train.shape[1],X_train.shape[2]), 1)

model = Sequential()
model.add(LSTM(32,activation="tanh", input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(Dropout(0.15))
model.add(Dense(1))
model.summary()
model.compile(optimizer="adam",loss="MSE")

# train LSTM model
model.compile(optimizer="adam",loss="MSE")
model.fit(X_train, y_train, epochs=10)

                        value
DateTime                     
2019-01-01 00:00:00  0.000000
2019-01-01 01:00:00  0.000000
2019-01-01 02:00:00  0.000008
2019-01-01 03:00:00  0.000000
2019-01-01 04:00:00  0.000008
...                       ...
2021-11-30 19:00:00  0.000000
2021-11-30 20:00:00  0.000000
2021-11-30 21:00:00  0.000000
2021-11-30 22:00:00  0.000000
2021-11-30 23:00:00  0.000000

[25560 rows x 1 columns]
                            value
DateTime                         
2019-01-01 00:00:00  0.000000e+00
2019-01-01 01:00:00  0.000000e+00
2019-01-01 02:00:00  1.753158e-08
2019-01-01 03:00:00  0.000000e+00
2019-01-01 04:00:00  1.753158e-08
...                           ...
2021-11-30 19:00:00  0.000000e+00
2021-11-30 20:00:00  0.000000e+00
2021-11-30 21:00:00  0.000000e+00
2021-11-30 22:00:00  0.000000e+00
2021-11-30 23:00:00  0.000000e+00

[25560 rows x 1 columns]
Solar Generation - EnerjiSA
(17826, 92, 1)
(7641, 92, 1)

Model: "sequential"
______________________________________

2022-12-16 00:44:02.491795: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-16 00:44:02.506329: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-16 00:44:02.506643: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-16 00:44:02.507215: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

KeyboardInterrupt: 

In [ ]:
pred = model.predict(X_test)
pred = scaler.inverse_transform(pred).reshape(1,-1)[0]
y_test_inv = scaler.inverse_transform(y_test).reshape(1,-1)[0]


def SMAPE(true, pred):
    return np.mean((abs(true - pred) / ((abs(true) + abs(pred))/2))*100)

def MDAPE(true, pred):
    return np.median((abs(true - pred) / true)*100)


print("MAE :", np.mean(abs(y_test_inv - pred)))
print("RMSE :", RMSE(y_test_inv,pred))
print("MSE :", MSE(y_test_inv,pred))
print("MAPE :", MAPE(y_test_inv,pred))
print("SMAPE :", SMAPE(y_test_inv,pred))
print("MDAPE :", MDAPE(y_test_inv,pred))
print("R2 :", r2_score(y_test_inv,pred))

In [ ]:
fig = go.Figure(go.Line(y = y_test_inv, name="original"))
fig.add_trace(go.Line(y = pred, name="predicted"))
fig

# Adversarial Attacks

In [ ]:
X_adversarial_fgsm = fgsm_attack(X_test, y_test, model, 0.01, np.inf, clip_min = 0, clip_max = 100)

pred = model.predict(X_adversarial_fgsm)
pred = scaler.inverse_transform(pred).reshape(1,-1)[0]
y_test_inv = scaler.inverse_transform(y_test).reshape(1,-1)[0]

print("MAE :", np.mean(abs(y_test_inv - pred)))
print("RMSE :", RMSE(y_test_inv,pred))
print("MSE :", MSE(y_test_inv,pred))
print("MAPE :", MAPE(y_test_inv[y_test_inv >= 10],pred[y_test_inv >= 10]))
print("SMAPE :", SMAPE(y_test_inv,pred))
print("R2 :", r2_score(y_test_inv,pred))

plt.plot(X_test[0], label = "original")
plt.plot(X_adversarial_fgsm.numpy()[0], label = "pertubated")
plt.legend()
plt.show()

In [ ]:
iterations = 10
alpha = 0.025
epsilon = 0.025
X_adversarial_pgd = pgd_attack(X_test, y_test, model, iterations, alpha, epsilon, np.inf, clip_min = 0, clip_max = 100)

pred = model.predict(X_adversarial_pgd)
pred = scaler.inverse_transform(pred).reshape(1,-1)[0]
y_test_inv = scaler.inverse_transform(y_test).reshape(1,-1)[0]

print("MAE :", np.mean(abs(y_test_inv - pred)))
print("RMSE :", RMSE(y_test_inv,pred))
print("MSE :", MSE(y_test_inv,pred))
print("MAPE :", MAPE(y_test_inv[y_test_inv >= 10],pred[y_test_inv >= 10]))
print("SMAPE :", SMAPE(y_test_inv,pred))
print("R2 :", r2_score(y_test_inv,pred))

plt.plot(X_test[0], label = "original")
plt.plot(X_adversarial_pgd.numpy()[0], label = "pertubated")
plt.legend()
plt.show()

In [ ]:
plt.plot(X_test[0], label = "original")
plt.plot(X_adversarial_fgsm.numpy()[0], label = "FGSM")
plt.plot(X_adversarial_pgd.numpy()[0], label = "PGD")
plt.legend()
plt.show()

In [ ]:
np.linalg.norm(X_test[0]-X_adversarial_fgsm.numpy()[0])

In [ ]:
np.linalg.norm(X_test[0]-X_adversarial_pgd.numpy()[0])

In [ ]:
np.linalg.norm(X_test[0]-X_test[0])

In [ ]:
inp = tf.convert_to_tensor(X_test, dtype = tf.float32)
imgv = tf.Variable(inp)
with tf.GradientTape() as tape:
    tape.watch(imgv)
    predictions = model(imgv)
    loss = tf.keras.losses.MeanSquaredError()(y_test, predictions)
    grads = tape.gradient(loss,imgv)

In [ ]:
fig,ax = plt.subplots()
# make a plot
ax1 = ax
ax2 = ax.twinx()

scaled_grad = (grads[0].numpy() - np.amin(grads[0].numpy())) / (np.amax(grads[0].numpy()) - np.amin(grads[0].numpy()))

# ax.plot(X_test[0], label = "original", color="red")
ax2.plot(tf.sign(grads[0]).numpy(), label = "signed", color="orange")
ax.plot(tf.sign(grads[0]).numpy()*scaled_grad, label = "signed", color="green")
plt.show()

# Cleverhans

In [ ]:
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method
from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent

In [ ]:
X_adversarial_fgsm = fast_gradient_method(model, X_test, 0.1, 1)

pred = model.predict(X_adversarial_fgsm)
pred = scaler.inverse_transform(pred).reshape(1,-1)[0]
y_test_inv = scaler.inverse_transform(y_test).reshape(1,-1)[0]

print("MAE :", np.mean(abs(y_test_inv - pred)))
print("RMSE :", RMSE(y_test_inv,pred))
print("MSE :", MSE(y_test_inv,pred))
print("MAPE :", MAPE(y_test_inv[y_test_inv >= 10],pred[y_test_inv >= 10]))
print("SMAPE :", SMAPE(y_test_inv,pred))
print("R2 :", r2_score(y_test_inv,pred))

plt.plot(X_test[0], label = "original")
plt.plot(X_adversarial_fgsm.numpy()[0], label = "pertubated")
plt.legend()
plt.show()